In [29]:
import PyKDL as kdl
import kdl_parser_py.urdf
import pickle
import torch
import sys, os
import cv2 as cv
import h5py
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import transformations as tf
import numpy as np
from itertools import repeat

np.set_printoptions(suppress=True)

%load_ext autoreload
%autoreload 2

util_path = os.path.abspath("../utils/")
sys.path.append(util_path)
import transformation as trans
import projections as proj
import robot_visualize as rbvis



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [70]:
act_path = "/home/zfei/code/act-plus-plus/"
sys.path.append(act_path)
from policy import EPACTPolicy, ACTPolicy
from brl_constants import TASK_CONFIGS
ckpt_path = "/mnt/data1/act/train_act_scara_3cam/3cam/"

config_path = ckpt_path + "config.pkl"
# step = 500000
# policy_model_path = ckpt_path + "policy_step_%i_seed_0.ckpt"%step
policy_model_path = ckpt_path + "policy_step_60000_seed_0.ckpt"

stats_path = ckpt_path + "dataset_stats.pkl"

with open(config_path, "rb") as f:
    policy_config = pickle.load(f)['policy_config']
    policy_config["ep_weight"] = 1.0
    print(policy_config)

policy = EPACTPolicy(policy_config)
# policy = ACTPolicy(policy_config)
policy_class = "EPACT"
# policy_class = "ACT"

with open(stats_path, "rb") as f:
    stats = pickle.load(f)

pre_process_qpos = (
    lambda s_qpos: torch.from_numpy((s_qpos - stats["qpos_mean"]) / stats["qpos_std"])
    .float()
    .cuda()
    .unsqueeze(0)
)
post_process_action = lambda a: a * stats["action_std"] + stats["action_mean"]

print("loading EPACT policy success")

{'lr': 1e-05, 'num_queries': 100, 'kl_weight': 10, 'hidden_dim': 512, 'dim_feedforward': 3200, 'lr_backbone': 1e-05, 'backbone': 'resnet18', 'enc_layers': 4, 'dec_layers': 7, 'nheads': 8, 'camera_names': ['wrist', 'wrist_down', 'top'], 'vq': False, 'vq_class': None, 'vq_dim': None, 'action_dim': 5, 'state_dim': 4, 'no_encoder': False, 'ep_weight': 1.0}
EPACT Args:  Namespace(lr=1e-05, lr_backbone=1e-05, batch_size=2, weight_decay=0.0001, epochs=300, lr_drop=200, clip_max_norm=0.1, backbone='resnet18', dilation=False, position_embedding='sine', camera_names=['wrist', 'wrist_down', 'top'], enc_layers=4, dec_layers=7, dim_feedforward=3200, hidden_dim=512, dropout=0.1, nheads=8, num_queries=100, pre_norm=False, masks=False, eval=False, onscreen_render=False, ckpt_dir='/mnt/data1/act/act_demo_z1_push_red/ckpt', policy_class='EPACT', task_name='act_demo_z1_push_red', seed=0, num_steps=2000, kl_weight=10, chunk_size=None, temporal_agg=False, use_vq=False, vq=False, vq_class=None, vq_dim=None,

/home/zfei/anaconda3/envs/py310/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/zfei/anaconda3/envs/py310/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Use end_pose_to_action:  True
Use VQ: False, None, None
number of parameters: 106.24M
EPACTPolicy ep_weight 1.0
KL Weight 10
loading EPACT policy success


In [71]:
camera_names = policy_config["camera_names"]
print(camera_names)

['wrist', 'wrist_down', 'top']


In [72]:
def pre_proccess_img(img):
    img_torch = torch.from_numpy(img).unsqueeze(0)
    img_torch = torch.einsum("k h w c -> k c h w", img_torch)
    img_torch = (img_torch / 255.0).float().cuda().unsqueeze(0)
    return img_torch

def pre_process_multi_img(curr_images):
    # a list of images [w, h, 3]
    curr_image = np.stack(curr_images, axis=0)
    img_torch = torch.from_numpy(curr_image)
    img_torch = torch.einsum("k h w c -> k c h w", img_torch)
    img_torch = (img_torch / 255.0).float().cuda().unsqueeze(0)
    return img_torch

def post_process_all_actions(all_actions):
    # post process all actions
    joint_states_traj = []
    actual_actions = []
    for i, raw_action in enumerate(all_actions):
        actual_action = post_process_action(raw_action)
        actual_actions.append(actual_action)
    actual_actions = np.asarray(actual_actions)
    # print(actual_actions)
    return actual_actions


In [73]:
act_path = "/home/zfei/code/act-plus-plus/"
sys.path.append(act_path)
from utils import EpisodicDataset
from utils import find_all_hdf5, flatten_list, get_norm_stats
from utils import load_data # data functions

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:

def forward_pass_epact(data, policy):
    image_data, qpos_data, action_data, is_pad, end_pose_data = data
    image_data, qpos_data, action_data, is_pad, end_pose_data = image_data.cuda(), qpos_data.cuda(), action_data.cuda(), is_pad.cuda(), end_pose_data.cuda()
    return policy(qpos_data, image_data, action_data, is_pad, end_pose_data) # TODO remove None

def forward_pass(data, policy):
    image_data, qpos_data, action_data, is_pad = data
    image_data, qpos_data, action_data, is_pad = image_data.cuda(), qpos_data.cuda(), action_data.cuda(), is_pad.cuda()
    return policy(qpos_data, image_data, action_data, is_pad) # TODO remove None

def repeater(data_loader):
    epoch = 0
    for loader in repeat(data_loader):
        for data in loader:
            yield data
        print(f'Epoch {epoch} done')
        epoch += 1

In [75]:
dataset_path_list = find_all_hdf5("/mnt/data1/act/train_act_scara_3cam/data1/", skip_mirrored_data=False)
norm_stats, all_episode_len = get_norm_stats(dataset_path_list)

camera_names = ['wrist', 'wrist_down', 'top']
train_episode_ids = [0, 1]
train_episode_len = [all_episode_len[i] for i in train_episode_ids]

val_episode_ids = [0, 1]
val_episode_len = [all_episode_len[i] for i in val_episode_ids]

chunk_size = 64


Found 51 hdf5 files


In [76]:
dataset_dir = "/mnt/data1/act/train_act_scara_3cam/data1/"

TASK_CONFIGS = {
        'epact':{
        'dataset_dir': dataset_dir + '/train_act_scara_3cam',
        'num_episodes': 100,
        'episode_len': 300,
        'camera_names': ['wrist', 'wrist_down', 'top']
    }
}

In [77]:
task_config = TASK_CONFIGS['epact']
name_filter = task_config.get('name_filter', lambda n: True)
batch_size_train = 8
batch_size_val = 8


In [79]:
chunk_size = 100
skip_mirrored_data = False
load_pretrain = False
train_ratio = 0.99
train_dataloader,val_dataloader, stats, _ = load_data(dataset_dir, 
                                                       name_filter, 
                                                       camera_names, 
                                                       batch_size_train, 
                                                       batch_size_val, 
                                                       chunk_size,
                                                       skip_mirrored_data,
                                                       load_pretrain,
                                                       policy_class,
                                                       stats_dir_l=None,
                                                       sample_weights=None,
                                                       train_ratio=train_ratio)
train_dataloader = repeater(train_dataloader)


Found 51 hdf5 files


Data from: ['/mnt/data1/act/train_act_scara_3cam/data1/']
- Train on [50] episodes
- Test on [1] episodes


Found 51 hdf5 files
Norm stats from: ['/mnt/data1/act/train_act_scara_3cam/data1/']
train_episode_len: [127, 124, 111, 168, 103, 137, 108, 121, 110, 117, 107, 125, 139, 121, 153, 135, 144, 118, 128, 110, 132, 165, 114, 104, 136, 154, 117, 140, 101, 118, 168, 129, 177, 175, 154, 123, 104, 90, 98, 151, 128, 118, 106, 120, 116, 124, 135, 116, 136, 161], val_episode_len: [135], train_episode_ids: [ 1 28 12 34 30  0  5  2 26 35  6 18 38 46 41 29 45 11 16 24 15 49 20  3
 14 10 48 50 27 13 19  9 44 25 40 43 47 21  7 32  4 33 22 36 39 23  8 42
 17 37], val_episode_ids: [31]
val data:  /mnt/data1/act/train_act_scara_3cam/data1/episode_28.hdf5
URDF Path:  /home/zfei/code/act-plus-plus/urdf/hitbot_model.urdf
kdl_parse urdf ok?:  True
augment_images:  False
Initializing transformations
URDF Path:  /home/zfei/code/act-plus-plus/urdf/hitbot_model.urdf
kdl_parse urdf ok?: 

In [80]:
data = next(train_dataloader)
forward_pass_epact(data, policy)



ValueError: too many values to unpack (expected 4)

In [67]:
print("image_data", image_data.shape)
print("qpos_data", qpos_data.shape)
print("action_data", action_data.shape)
print("is_pad", is_pad.shape)

image_data torch.Size([8, 3, 3, 480, 640])
qpos_data torch.Size([8, 4])
action_data torch.Size([8, 100, 5])
is_pad torch.Size([8, 100])


In [68]:
policy.model.num_queries

100

In [69]:
policy(qpos_data, image_data, action_data, is_pad)

image torch.Size([8, 3, 3, 480, 640])
qpos torch.Size([8, 4])
actions torch.Size([8, 100, 5])
is_pad torch.Size([8, 100])


{'l1': tensor(0.7795, device='cuda:0', grad_fn=<MeanBackward0>),
 'kl': tensor(7.9838, device='cuda:0', grad_fn=<SelectBackward0>),
 'loss': tensor(80.6176, device='cuda:0', grad_fn=<AddBackward0>)}

In [13]:
h5data_file = "/mnt/data1/act/train_act_scara_3cam/data1/episode_30.hdf5"

with torch.inference_mode():
    with h5py.File(h5data_file, "r") as root:
        index = 0
        qpos_np = root["/observations/qpos"][index]
        img_wrist_up_np = root["/observations/images/wrist"][index]
        img_wrist_down_np = root["/observations/images/wrist_down"][index]
        img_top_np = root["/observations/images/top"][index]

        qpos_all =  root["/observations/qpos"][()]

        curr_images = [img_wrist_up_np, img_wrist_down_np, img_top_np]

        action_obs_np = root["action"][()]
        qpos = pre_process_qpos(qpos_np)
        curr_images_torch = pre_process_multi_img(curr_images)
        # print("qpos: ",qpos.shape)
        all_actions, all_end_poses = policy(qpos, curr_images_torch)
        all_actions = all_actions.squeeze(0).cpu().numpy()

        print("all_actions: ", all_actions.shape)
        print("action_obs_np: ", action_obs_np.shape)

        # actual_action = post_process_action(raw_action)
        # print("actual action %i"%index, actual_action)


action_obs_np = action_obs_np[index:]
qpos_all = np.asarray(qpos_all)
qpos_all = qpos_all[index:]


all_actions:  (100, 5)
action_obs_np:  (127, 5)
